In [1]:
#Load in relevant libraries
import math

import numpy as np

from astropy import units as u
from astropy import coordinates as coord
from astropy.coordinates import SkyCoord, ICRS, Galactocentric, Distance, LSR
from astropy.coordinates import FK5
from astropy.coordinates import CartesianRepresentation, CartesianDifferential
from astropy.table import Table, vstack, hstack
from astropy.table import Column
from astropy.io import ascii

import gala.potential as gp
import gala.dynamics as gd
from gala.units import galactic

from scipy import stats

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler, normalize

import time

import matplotlib.pyplot as plt

import pandas as pd

In [2]:
#Load in the dataset which contains all the halo stars and their corresponding DBSCAN labels
t = Table.read('Step_3_Output.fits')

#Also set up the table as the initial survivor dataset
t.write('Step_6_Survivor.fits', overwrite = True)

print("Total Number of Stars in Data Set: " + str(len(t)))

Total Number of Stars in Data Set: 6789


In [3]:
#These variables stipulate the sample size and the number of rounds the clusters need
# to survive. Generally you only want to create only one sample size for each run, 
#  although you can change the number of iterations.
sample_size = 1
iterations = 2

#This line of code creates repeated challenger sets which get compared to the actual
# clusters. 

#We need to convert the table to a pandas table as we have to use the monte-carlo
# iterations.
star_list = t.to_pandas()


#Set the acceptance percentage which allows for a cluster to have survived
acc_perc = 50 

 
for N in range(0,iterations):
    
    Monte_Vals = Table()
    #The program informs the user which round is underway
    print("--Changellener Set " + str(N+1) + " has started--\n")
    #print('\n')
    
    #This monte-carlo algorythm creates a mirror subset, exactly the same way as before
    for star_num,star in star_list.iterrows():

        tbl_names = ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'radial_velocity']

        C = np.zeros((6, 6))

        for i,name in enumerate(tbl_names):

            err_name = "{0}_error".format(name)

            C[i,i] = (star[err_name]) ** 2

            for i,name1 in enumerate(tbl_names):

                for j,name2 in enumerate(tbl_names):

                    if j <= i: # we make it symmetric below

                        continue

                    full_name = "{0}_{1}_corr".format(name1, name2)

                    if full_name not in star_list.columns:

                        continue

                    C[i,j] = star[full_name] * np.sqrt(C[i,i]*C[j,j])

                    C[j,i] = star[full_name] * np.sqrt(C[i,i]*C[j,j])



        y = np.array([star.ra, star.dec, star.parallax, star.pmra, star.pmdec, star.radial_velocity])

        samples = np.random.multivariate_normal(y, C, size=sample_size)



        coord_samples = coord.ICRS(ra=samples[:,0] * u.deg,

                                   dec=samples[:,1] * u.deg,

                                   distance=1000./samples[:,2] * u.pc,

                                   pm_ra_cosdec=samples[:,3] * u.mas/u.yr,

                                   pm_dec=samples[:,4] * u.mas/u.yr,

                                   radial_velocity=samples[:,5] * u.km/u.s)

        galc = coord_samples.transform_to(Galactocentric)

        T = Table()

        STAR = Column(np.repeat(star['source_id'], sample_size),'source_id')
        T.add_column(STAR)

        RA = Column(coord_samples.ra, 'ra')
        DEC = Column(coord_samples.dec, 'dec')
        DIST = Column(coord_samples.distance, 'dist')
        PMRA = Column(coord_samples.pm_ra_cosdec, 'pmra')
        PMDEC = Column(coord_samples.pm_dec, 'pmdec')
        RV = Column(coord_samples.radial_velocity, 'radial_velocity')

        T.add_column(RA)
        T.add_column(DEC)
        T.add_column(DIST)
        T.add_column(PMRA)
        T.add_column(PMDEC)
        T.add_column(RV)

        x = galc.x
        y = galc.y
        z = galc.z

        x = Column(x,name='x_val')
        y = Column(y,name='y_val')
        z = Column(z,name='z_val')

        U = galc.v_x
        V = galc.v_y
        W = galc.v_z

        U = Column(U,name='U')
        V = Column(V,name='V')
        W = Column(W,name='W')

        T.add_column(x)
        T.add_column(y)
        T.add_column(z)

        T.add_column(U)
        T.add_column(V)
        T.add_column(W)

        Monte_Vals = vstack([Monte_Vals,T])
        
        print("Monte Carlo Stars Created: " + str(star_num) + " out of: " + str(len(star_list)), end="\r")

    step = len(star_list)
    ACT = Table()
    
    #The actions of this set also need to be calculated
    for i in range(0,sample_size):    
        #Taking the cartesian position and velocity coordinates from the table and saving them as vectors
        x = Monte_Vals['x_val'][step*i:step*(1+i)]
        y = Monte_Vals['y_val'][step*i:step*(1+i)]
        z = Monte_Vals['z_val'][step*i:step*(1+i)]
        U = Monte_Vals['U'][step*i:step*(1+i)]
        V = Monte_Vals['V'][step*i:step*(1+i)]
        W = Monte_Vals['W'][step*i:step*(1+i)]

        T = Table()

        #In order to calculate the actions, you need to specifiy a graviational potential. 
        # In this case we use a combination of potentials in the same manner as Helmi et al 2017. 
        #  Using the MilkyWayPotential class models the buldge, disk and halo in combination. 
        pot = gp.MilkyWayPotential(units=galactic)

        #We use the Gala function PhaseSpacePosition to save the 6-D phase space coordinates, which
        # are taken from the above vectors,
        w0 = gd.PhaseSpacePosition(pos=[x, y, z]*u.pc,
                               vel=[U, V, W]*u.km/u.s)
        #The orbit of the star is then calculated using the following command for 1000 steps
        orbit = gp.Hamiltonian(pot).integrate_orbit(w0, dt=1., n_steps=1000)

        #First Calculate the energies of each orbit
        E = orbit.energy().to(u.km*u.km/(u.s*u.s))
        Energy = Column(E[0], name = 'Energy')
        #trunc.add_column(Energy)

        #Z-coomponent of angular momentum is calculated,
        Lz = orbit.angular_momentum()[2].to(u.km*u.kpc/u.s)
        L_z = Column(Lz[0], name = 'Lz')
        #trunc.add_column(L_z)

        #The x and y components of angular momentum are calculated and combined using pythagoras theorem
        Lx = orbit.angular_momentum()[0].to(u.km*u.kpc/u.s)
        Ly = orbit.angular_momentum()[1].to(u.km*u.kpc/u.s)

        #We calculcualte the poerpedicular component of the angular momentum
        Lp = np.sqrt(Lx[0]*Lx[0]+Ly[0]*Ly[0])


        L_perp = Column(Lp, name = 'L_perp')

        T.add_column(Energy)
        L_z=-L_z
        T.add_column(L_z)
        T.add_column(L_perp)
        ACT = vstack([ACT,T])

        print("Completed Iterations: " + str(i) + " out of: " + str(sample_size), end="\r")

    Monte_Vals = hstack([Monte_Vals,ACT])

    t_sphere = Monte_Vals

    #This line of code coverts any two columns specified from the data set in a 2-D
    # array which is compatable with the DBSCAN algorithm.
    x_val='Lz'
    y_val='L_perp'
    z_val='Energy'

    A=t_sphere[x_val][0]
    B=t_sphere[y_val][0]
    C=t_sphere[z_val][0]
    X=[A,B,C]
    for i in range(1,len(t_sphere)):
        D=t_sphere[x_val][i]
        E=t_sphere[y_val][i]
        F=t_sphere[z_val][i]
        G=np.array([D,E,F])
        X=np.vstack([X,G])

        #X=normalize(X)
    X = StandardScaler().fit_transform(X)

    #This block of code runs the DBSCAN and informs the user how many clusters
    # were in fact detected.
    EPS = 0.328

    # Compute DBSCAN
    db = DBSCAN(eps=EPS, min_samples=4).fit(X)
    labels_true = np.full((1000,1),1000)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    
    print('Estimated number of clusters: ' + str(n_clusters_))

    #The labels for each star in the set are extracted and then converted into 
    # a column.
    P = db.labels_

    #Creates into a column which can be added to the table
    label = Column(P,name='cluster_label')

    #t_sphere.remove_column('cluster_label') #if you need to add the column
    t_sphere.add_column(label) #if you need to add the column
    t_sphere.write('Step_6_Varied_Monte_Vals.fits',overwrite=True)

    #The program will save the surviving clusters as a seperate file which gets re-loaded
    # into the program periodically to surive against additional challengers
    t_survivor = Table.read('Step_6_Survivor.fits')
    t_survivor_i = Table()
    
    print("\nChecking which clusters have survived: " + str(N+1)+'\n')
    for k in range(1,np.ndarray.max(t_survivor['cluster_label'])+1):
        t_sub = t_survivor[(t_survivor['cluster_label'])==k]
        
        for m in range(1,np.ndarray.max(t_sphere['cluster_label'])+1):
            t_sphere_sub = t_sphere[(t_sphere['cluster_label'])==m]

            n = 0
            for i in range(0,len(t_sub)):
                c = t_sub['source_id'][i]
                for j in range(0,len(t_sphere_sub)):
                    c_sphere = t_sphere_sub['source_id'][j]


                    if c == c_sphere:
                        #print(c_sphere)
                        n = n + 1

            if n > 0:

                if (n/len(t_sub))*100 > acc_perc:
                    print("Cluster " + str(t_sub['cluster_label'][0]) + " surivived")

                    t_survivor_i = vstack([t_survivor_i,t_sub])

    print("\n--Changellener Set " + str(N+1) + " has ended--")
    print()
    
    t_survivor_i.write('Step_6_Survivor.fits', overwrite = True)    

--Changellener Set 1 has started--

Estimated number of clusters: 9 1ut of: 6789

Checking which clusters have survived: 1

Cluster 1 surivived
Cluster 2 surivived
Cluster 4 surivived
Cluster 6 surivived
Cluster 7 surivived
Cluster 9 surivived

--Changellener Set 1 has ended--

--Changellener Set 2 has started--

Estimated number of clusters: 111ut of: 6789

Checking which clusters have survived: 2

Cluster 1 surivived
Cluster 2 surivived
Cluster 4 surivived
Cluster 6 surivived

--Changellener Set 2 has ended--

